# 08 - Portfolio with Option Overlay

Capstone: compare unhedged underlying exposure against a budget-constrained protective put overlay.

In [4]:
%cd /content
!rm -rf interactive_portfolio_optimization
!git clone https://github.com/basarr/interactive_portfolio_optimization.git
!ls /content/interactive_portfolio_optimization

/content
Cloning into 'interactive_portfolio_optimization'...
remote: Enumerating objects: 117, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 117 (delta 53), reused 44 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (117/117), 69.97 KiB | 2.26 MiB/s, done.
Resolving deltas: 100% (53/53), done.
data  notebooks  pyproject.toml  README.md  results  sources  src  tests


In [5]:
from pathlib import Path
import sys

ROOT = Path("/content/interactive_portfolio_optimization")
if not (ROOT / "src").exists():
    raise FileNotFoundError(f"Bad ROOT: {ROOT}")

for p in [ROOT/"results", ROOT/"results"/"tables", ROOT/"results"/"figures", ROOT/"results"/"logs", ROOT/"results"/"reports"]:
    p.mkdir(parents=True, exist_ok=True)

if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

print("ROOT OK:", ROOT)


ROOT OK: /content/interactive_portfolio_optimization


In [6]:
import pandas as pd

from src.config import config_dict
from src.portfolio_overlay import protective_put_overlay_simulation, overlay_risk_metrics
from src.plotting import plot_overlay_distribution

cfg = config_dict(fast_mode=True)


In [7]:
overlay = protective_put_overlay_simulation(
    S0=cfg['S0'],
    r=cfg['R'],
    q=cfg['Q'],
    sigma=cfg['SIGMA'],
    T_sim=cfg['T'],
    K_put=cfg['PROTECTIVE_PUT_STRIKE'],
    T_put=cfg['PUT_MATURITY'],
    notional=cfg['PORTFOLIO_NOTIONAL'],
    premium_budget_fraction=cfg['HEDGE_BUDGET_FRACTION'],
    n_paths=40_000,
    n_steps=252,
    seed=cfg['SEED'],
)

metrics = overlay_risk_metrics(overlay['unhedged_returns'], overlay['hedged_returns'])
metrics['premium_paid'] = [overlay['premium_paid'], overlay['premium_paid']]
metrics['coverage_ratio'] = [overlay['coverage_ratio'], overlay['coverage_ratio']]
metrics.to_csv(ROOT / 'results' / 'tables' / 'overlay_metrics.csv')

plot_overlay_distribution(
    overlay['unhedged_returns'],
    overlay['hedged_returns'],
    ROOT / 'results' / 'figures' / 'overlay_return_dist.png',
)

metrics


,mean,std,annualized_vol,cvar_95,max_drawdown,premium_paid,coverage_ratio
unhedged,0.020384,0.206658,3.280592,-0.336003,-1.0,17398.426858,1.0
protective_put,0.020647,0.196725,3.122912,-0.312721,-1.0,17398.426858,1.0


# Notebook 08 — Protective Put Overlay

We simulate a protective put overlay on a long underlying position. The portfolio holds the asset and buys puts with strike $K_{\text{put}}$, subject to a premium budget. Terminal return is

$R = \frac{V_T}{V_0} - 1 = \frac{\text{underlying value} - \text{premium} + \text{put payoff}}{\text{notional}} - 1$.

Results

Unhedged:
mean $\approx 0.02038$,
std $\approx 0.20666$,
annualized vol $\approx 3.2806$,
CVaR$_{95} \approx -0.3360$.

Protective put:
mean $\approx 0.02065$,
std $\approx 0.19673$,
annualized vol $\approx 3.1229$,
CVaR$_{95} \approx -0.3127$.

Interpretation

The protective put reduces volatility and left-tail risk (lower std and CVaR).
The distribution shows a tighter left tail, indicating fewer extreme losses.
Mean return remains similar because the premium budget is small; larger protection would generally reduce expected return.

Note on max drawdown

The reported value of $-1.0$ is not informative here. We simulate single-period terminal returns, whereas max drawdown requires a multi-period path. For this notebook, the relevant metrics are mean, std, and CVaR.